In [1]:
#load the saved matrices
import pickle
train_X=pickle.load(open( "../dipep_new_train_x.p", "rb" ) )
test_X=pickle.load(open( "../dipep_new_test_x.p", "rb" ) )
train_Y=pickle.load(open( "../dipep_new_train_y.p", "rb" ) )
test_Y=pickle.load(open( "../dipep_new_test_y.p", "rb" ) )
mean_arr=pickle.load(open( "../dipep_new_mean.p", "rb" ) )
mean_std=pickle.load(open( "../dipep_new_std.p", "rb" ) )

In [ ]:
#import keras and numpy
import numpy
from scipy import linalg as LA
from scipy.optimize import fminbound as OPT
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
import keras


In [ ]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
f1=open('./testfile', 'w+')

In [ ]:
def evaluate_f(lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2, vv=0):
    #opt=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    opt=SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=False)
    model = Sequential()
    model.add(Dense(402, input_dim=402, kernel_initializer='random_uniform',activation='relu'))
    model.add(Dropout(dd1))
    model.add(Dense(200,activation='relu'))
    model.add(Dropout(dd2))
    model.add(Dense(200,activation='relu'))
    model.add(Dropout(dd3))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist=model.fit(train_X, train_Y,verbose=vv, epochs=50, batch_size=200)
    #scores = model.evaluate(test_X, test_Y, verbose=vv)
    arr_pred=model.predict(test_X)
    #if vv:
    #    print("Accuracy: %.2f%%" % (scores[1]*100))
    keras.backend.clear_session()
    return LA.norm(arr_pred-test_Y)
    #return scores[1]
    #return hist;
    
def evaluate_f_v(VV):
    val=evaluate_f(lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3], vv=0)
    return val
        

In [ ]:
def evaluate_f_acc(lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2, vv=0):
    #opt=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    opt=SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=False)
    model = Sequential()
    model.add(Dense(402, input_dim=402, kernel_initializer='random_uniform',activation='relu'))
    model.add(Dropout(dd1))
    model.add(Dense(200,activation='relu'))
    model.add(Dropout(dd2))
    model.add(Dense(200,activation='relu'))
    model.add(Dropout(dd3))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist=model.fit(train_X, train_Y,verbose=vv, epochs=50, batch_size=200)
    scores = model.evaluate(test_X, test_Y, verbose=vv)
    #arr_pred=model.predict(test_X)
    #if vv:
    #    print("Accuracy: %.2f%%" % (scores[1]*100))
    keras.backend.clear_session()
    #return LA.norm(arr_pred-test_Y)
    return scores[1]
    #return hist;
    
def evaluate_f_acc_v(VV):
    val=evaluate_f_acc(lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3], vv=0)
    return val
        

In [ ]:
def avg_evaluate_f(lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2, vv=0):
    f1=evaluate_f(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f2=evaluate_f(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f3=evaluate_f(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f4=evaluate_f(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f5=evaluate_f(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    return numpy.average([f1,f2,f3,f4,f5])

def avg_evaluate_f_v(VV):
    val=avg_evaluate_f(lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3], vv=0)
    return val

def avg_evaluate_f_acc(lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2, vv=0):
    f1=evaluate_f_acc(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f2=evaluate_f_acc(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f3=evaluate_f_acc(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f4=evaluate_f_acc(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    f5=evaluate_f_acc(lr=lr, dd1=dd1, dd2=dd3, dd3=dd3, vv=vv)
    return numpy.average([f1,f2,f3,f4,f5])

def avg_evaluate_f_acc_v(VV):
    val=avg_evaluate_f_acc(lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3], vv=0)
    return val

In [ ]:
def evaluate_G(lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2,h=0.001, k=0.0001):
    df_lr=((avg_evaluate_f(lr=lr+k,dd1=dd1, dd2=dd2, dd3=dd3, vv=0)-
            avg_evaluate_f(lr=lr-k,dd1=dd1, dd2=dd2, dd3=dd3, vv=0))/(2*k))
    df_dd1=((avg_evaluate_f(dd1=dd1+h,dd2=dd2, dd3=dd3, vv=0,lr=lr)-
             avg_evaluate_f(dd1=dd1-h,dd2=dd2, dd3=dd3, vv=0,lr=lr))/(2*h))
    df_dd2=((avg_evaluate_f(dd2=dd2+h,dd1=dd1, dd3=dd3, vv=0,lr=lr)-
             avg_evaluate_f(dd2=dd2-h,dd1=dd1, dd3=dd3, vv=0,lr=lr))/(2*h))
    df_dd3=((avg_evaluate_f(dd3=dd3+h,dd1=dd1, dd2=dd2, vv=0,lr=lr)-
             avg_evaluate_f(dd3=dd3-h,dd1=dd1, dd2=dd2, vv=0,lr=lr))/(2*h))
    G=numpy.array([df_lr,df_dd1,df_dd2,df_dd3],dtype=numpy.float)
    return G

def evaluate_G_v(VV):
    val=evaluate_G(lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3])
    return val

In [ ]:
def evaluate_H(ff,lr=0.001, dd1=0.2, dd2=0.2, dd3=0.2,h=0.001, k=0.0001,vv=0):
    
    df_lr_lr=((avg_evaluate_f(lr=lr+2*k, dd1=dd1, dd2=dd2, dd3=dd3,vv=vv)-2*ff+
               avg_evaluate_f(lr=lr-2*k, dd1=dd1, dd2=dd2, dd3=dd3,vv=vv))/(4*k*k))
    df_dd1_dd1=((avg_evaluate_f(lr=lr, dd1=dd1+2*h, dd2=dd2, dd3=dd3,vv=vv)-2*ff+
               avg_evaluate_f(lr=lr, dd1=dd1-2*h, dd2=dd2, dd3=dd3,vv=vv))/(4*h*h))
    df_dd2_dd2=((avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2+2*h, dd3=dd3,vv=vv)-2*ff+
               avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2-2*h, dd3=dd3,vv=vv))/(4*h*h))
    df_dd3_dd3=((avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2, dd3=dd3+2*h,vv=vv)-2*ff+
               avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2, dd3=dd3-2*h,vv=vv))/(4*h*h))
    
    df_lr_dd1=((avg_evaluate_f(lr=lr+k, dd1=dd1+h, dd2=dd2, dd3=dd3,vv=vv)-
                avg_evaluate_f(lr=lr+k, dd1=dd1-h, dd2=dd2, dd3=dd3,vv=vv)-
                avg_evaluate_f(lr=lr-k, dd1=dd1+h, dd2=dd2, dd3=dd3,vv=vv)+
                avg_evaluate_f(lr=lr-k, dd1=dd1-h, dd2=dd2, dd3=dd3,vv=vv))/(4*k*h))
    df_lr_dd2=((avg_evaluate_f(lr=lr+k, dd1=dd1, dd2=dd2+h, dd3=dd3,vv=vv)-
                avg_evaluate_f(lr=lr+k, dd1=dd1, dd2=dd2-h, dd3=dd3,vv=vv)-
                avg_evaluate_f(lr=lr-k, dd1=dd1, dd2=dd2+h, dd3=dd3,vv=vv)+
                avg_evaluate_f(lr=lr-k, dd1=dd1, dd2=dd2-h, dd3=dd3,vv=vv))/(4*k*h))
    df_lr_dd3=((avg_evaluate_f(lr=lr+k, dd1=dd1, dd2=dd2, dd3=dd3+h,vv=vv)-
                avg_evaluate_f(lr=lr+k, dd1=dd1, dd2=dd2, dd3=dd3-h,vv=vv)-
                avg_evaluate_f(lr=lr-k, dd1=dd1, dd2=dd2, dd3=dd3+h,vv=vv)+
                avg_evaluate_f(lr=lr-k, dd1=dd1, dd2=dd2, dd3=dd3-h,vv=vv))/(4*k*h))
    
    df_dd1_dd2=((avg_evaluate_f(lr=lr, dd1=dd1+h, dd2=dd2+h, dd3=dd3,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1+h, dd2=dd2-h, dd3=dd3,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1-h, dd2=dd2+h, dd3=dd3,vv=vv)+
                 avg_evaluate_f(lr=lr, dd1=dd1-h, dd2=dd2-h, dd3=dd3,vv=vv))/(4*h*h))
    df_dd1_dd3=((avg_evaluate_f(lr=lr, dd1=dd1+h, dd2=dd2, dd3=dd3+h,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1+h, dd2=dd2, dd3=dd3-h,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1-h, dd2=dd2, dd3=dd3+h,vv=vv)+
                 avg_evaluate_f(lr=lr, dd1=dd1-h, dd2=dd2, dd3=dd3-h,vv=vv))/(4*h*h))
    
    df_dd2_dd3=((avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2+h, dd3=dd3+h,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2+h, dd3=dd3-h,vv=vv)-
                 avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2-h, dd3=dd3+h,vv=vv)+
                 avg_evaluate_f(lr=lr, dd1=dd1, dd2=dd2-h, dd3=dd3-h,vv=vv))/(4*h*h))
    
    
    H=numpy.array([[df_lr_lr ,df_lr_dd1 ,df_lr_dd2 ,df_lr_dd3],
                   [df_lr_dd1,df_dd1_dd1,df_dd1_dd2,df_dd1_dd3],
                   [df_lr_dd2,df_dd1_dd2,df_dd2_dd2,df_dd2_dd3],
                   [df_lr_dd3,df_dd1_dd3,df_dd2_dd3,df_dd3_dd3]],dtype=numpy.float)
    #HH=numpy.reshape(H,(4,4))

    return H

def evaluate_H_v(ff,VV):
    val=evaluate_H(ff,lr=VV[0], dd1=VV[1], dd2=VV[2], dd3=VV[3], vv=0)
    return val

In [ ]:
def q_model(pp,ff,GG,HH):
    RR=ff+pp@GG +(0.5*(pp@HH@pp))
    return RR

In [ ]:
def sub_vec(mu,stp_dir,full_step):
    return mu[0]*stp_dir+mu[1]*full_step

In [ ]:
def solve_sub_space(stp,fu,HH,GG):
    A=numpy.array([[stp @ HH @ stp, stp @ HH @ fu],
                   [fu  @ HH @ stp, fu  @ HH @ fu]],dtype=numpy.float)
    b=numpy.array([stp @GG ,fu @ GG],dtype=numpy.float)
    return -1*numpy.linalg.solve(A,b)

In [ ]:
def one_search(sigma,vec1,vec2,ff,GG,HH,rad):
    kk=rad*(numpy.cos(sigma)*vec1+numpy.sin(sigma)*vec2)
    return q_model(kk,ff,GG,HH)

In [ ]:
def one_vec(sigma,vec1,vec2,rad):
    kk=rad*(numpy.cos(sigma)*vec1+numpy.sin(sigma)*vec2)
    return kk

In [ ]:
def change_base(stp,fu):
    q1=stp/LA.norm(stp)
    inn=fu-numpy.inner(stp,fu)*q1
    q2=inn/LA.norm(inn)
    
    return (q1,q2)

In [ ]:
def condition(beta,A):
    I=numpy.identity(4)
    HH=A+beta*I
    return numpy.linalg.cond(HH)


In [ ]:
path = numpy.empty((0,4), dtype=numpy.float)
path_grad=numpy.empty((0,4), dtype=numpy.float)
zero_arr=numpy.array([0,0,0,0])
rad=0.001
max_rad=0.1
tol=1/4
GG=1
XX=numpy.array([0.001,0.2,0.2,0.2])
tt=1
rho_count=0
#cat_n.reshape((1,cat_n.shape[0]))
path = numpy.append(path,XX.reshape((1,XX.shape[0])) , axis=0)
while ((LA.norm(GG)>0.0001) and (tt<40)):
    print('itt= ',tt,file=f1)
    print('evaluating function',file=f1)
    FF=avg_evaluate_f_v(XX)
    print('done ',FF,file=f1)
    print('evaluating gradient',file=f1)
    GG=evaluate_G_v(XX)
    path_grad=numpy.append(path_grad,GG.reshape((1,GG.shape[0])) , axis=0)
    if(LA.norm(GG)<0.0001):
        break
    print('evaluating hessian',file=f1)
    HH=evaluate_H_v(FF,XX)
    print('done',file=f1)
    print('getting eigenvalues',file=f1)
    min_e=numpy.real(numpy.min(LA.eigvals(HH)))
    
    print('min eig =',  min_e,file=f1)
    if min_e < 0:
        (kk1,kk2,kk3,kk4)=OPT(condition,-1.1*min_e,-2*min_e,args=(HH,),full_output=1)
        print(kk1,kk2,kk3,kk4)
        print(kk1/min_e)
        HH=HH+kk1*numpy.identity(4)
    print('solving subspace',file=f1)
    full_step=-1*numpy.linalg.solve(HH,GG)
    stp_dir=-1*((GG @ GG)/(GG @ HH @ GG))*GG
    kk=solve_sub_space(stp_dir,full_step,HH,GG)
    pp=sub_vec(kk,stp_dir,full_step)
    t_pp=XX+pp
    (q1,q2)=change_base(stp_dir,full_step)
    if ((LA.norm(pp)>rad) or (numpy.min(t_pp)<0) or (numpy.max(t_pp[1:])>1)):
        dic= (q1,q2,FF,GG,HH,rad)
        (opt1,opt2,opt3,opt4)=OPT(one_search,0,2*numpy.pi,args=dic,full_output=1)
        pp=one_vec(opt1,q1,q2,rad)
    FF_n=avg_evaluate_f_v(XX+pp)
    rho_top=FF-FF_n
    rho_bot=q_model(zero_arr,FF,GG,HH)-q_model(pp,FF,GG,HH)
    if (rho_bot<10*numpy.finfo(float).eps):
        print('rho_bot too close to epsilon',file=f1)
        rad=0.001
        rho_count=rho_count+1
        continue
    rho=rho_top/rho_bot
    while ((rho < 1/4) or (LA.norm(pp)>rad) or (numpy.min(t_pp)<0) or (numpy.max(t_pp[1:])>1)):
        rad=rad*1/4
        dic= (q1,q2,FF,GG,HH,rad)
        (opt1,opt2,opt3,opt4)=OPT(one_search,0,2*numpy.pi,args=dic,full_output=1)
        pp=one_vec(kk1,q1,q2,rad)
        FF_n=avg_evaluate_f_v(XX+pp)
        rho_top=FF-FF_n
        rho_bot=q_model(zero_arr,FF,GG,HH)-q_model(pp,FF,GG,HH)
        if (rho_bot<10*numpy.finfo(float).eps):
            print('rho_bot too close to epsilon',file=f1)
            rad=0.001
            rho_count=rho_count+1
            break
        rho=rho_top/rho_bot
    if ((rho>3/4) and (LA.norm(pp)<1.001*rad)):
        rad=min(2*rad,max_rad)
    XX=XX+pp
    tt=tt+1
    print('final radius ',rad,file=f1)
    print('grad norm ',LA.norm(GG),file=f1)
    path = numpy.append(path,XX.reshape((1,XX.shape[0])) , axis=0)
    

In [ ]:
print(path)
print(path_grad)

In [ ]:
import pickle
pickle.dump(path, open( "path.p", "wb" ) )
pickle.dump(path_grad, open( "path_grad.p", "wb" ) )

In [ ]:
XX2=numpy.array([0.001,0.2,0.2,0.2])
#GG_i=GG

In [ ]:
%%time
FF=avg_evaluate_f_v(XX2)

In [ ]:
%%time
GG_i=evaluate_G_v(XX2)

In [ ]:
%%time
HH=evaluate_H_v(FF,XX2)

In [ ]:
#from keras.models import load_model
#model = load_model('di_new_model.h5')

In [ ]:
#arr_pred=model.predict(test_X)
#numpy.set_printoptions(formatter={'float': '{: 0.3f}'.format})

In [ ]:
#print(arr_pred-test_Y)



In [ ]:
XX=pickle.load(open( "2_path.p", "rb" ) )

In [ ]:
print(XX[0,...],' -> ',avg_evaluate_f_acc_v(XX[0,...]))
print(XX[29,...],' -> ',avg_evaluate_f_acc_v(XX[29,...]))